# 卷积神经网络做mnist识别

In [5]:
import pandas as pd
import numpy as np

In [15]:
#实例化一个小型卷积神经网络来进行手写数字识别
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),input_shape=(28,28,1),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


可以看到每一个卷积层和池化层的输出都是3D张量。

In [16]:
model.add(layers.Flatten())
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(10,activation = 'softmax'))

In [ ]:
我们将进行10类别分类，最后一层使用带有十个输出的softmax函数激活，网络架构如下：

In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [6]:
data = np.load('D:\\data\\mnist.npz')

In [7]:
data.files

['x_test', 'x_train', 'y_train', 'y_test']

In [9]:
x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']

In [21]:
train_images = x_train.reshape((60000,28,28,1))
train_images = train_images.astype('float32') / 255

test_images = x_test.reshape((10000,28,28,1))
test_images = test_images.astype('float32') / 255

In [24]:
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)

In [25]:
model.compile(optimizer='rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
model.fit(train_images,train_labels,epochs = 5,batch_size = 64)

Epoch 1/5
938/938 [==============================] - 35s 38ms/step - loss: 0.1618 - accuracy: 0.9493
Epoch 2/5
938/938 [==============================] - 32s 34ms/step - loss: 0.0460 - accuracy: 0.9858
Epoch 3/5
938/938 [==============================] - 31s 33ms/step - loss: 0.0323 - accuracy: 0.99000s - loss: 0.0323 - 
Epoch 4/5
938/938 [==============================] - 31s 33ms/step - loss: 0.0241 - accuracy: 0.9927
Epoch 5/5
938/938 [==============================] - 32s 34ms/step - loss: 0.0186 - accuracy: 0.9944


In [26]:
test_loss,test_acc = model.eva

luate(test_images,test_labels)
test_acc

313/313 [==============================] - 1s 4ms/step - loss: 0.0421 - accuracy: 0.9888


0.9887999892234802

可以看到测试精度达到了98.88%，是相当不错的一个效果。

## 利用torch搭建CNN

In [2]:
import torch 
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import mnist
from torch import  nn
from torch.autograd import Variable
from torch import  optim
from torchvision import transforms
 
# 定义CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(
                nn.Conv2d(1,16,kernel_size=3), # 16, 26 ,26
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True))
        
        self.layer2 = nn.Sequential(
                nn.Conv2d(16,32,kernel_size=3),# 32, 24, 24
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2)) # 32, 12,12     (24-2) /2 +1
        
        self.layer3 = nn.Sequential(
                nn.Conv2d(32,64,kernel_size=3), # 64,10,10
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True))
        
        self.layer4 = nn.Sequential(
                nn.Conv2d(64,128,kernel_size=3),  # 128,8,8
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2))  # 128, 4,4
        
        self.fc = nn.Sequential(
                nn.Linear(128 * 4 * 4,1024),
                nn.ReLU(inplace=True),
                nn.Linear(1024,128),
                nn.ReLU(inplace=True),
                nn.Linear(128,10))
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        
        return x
 
 
# 使用内置函数下载mnist数据集
train_set = mnist.MNIST('./data',train=True,download=True)
test_set = mnist.MNIST('./data',train=False)
 
# 预处理=>将各种预处理组合在一起
data_tf = transforms.Compose(
                [transforms.ToTensor(),
                 transforms.Normalize([0.5],[0.5])])
 
train_set = mnist.MNIST('./data',train=True,transform=data_tf,download=True)
test_set = mnist.MNIST('./data',train=False,transform=data_tf,download=True)
 
train_data = DataLoader(train_set,batch_size=64,shuffle=True)
test_data = DataLoader(test_set,batch_size=128,shuffle=False)
 
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),1e-1)
 
nums_epoch = 20
 
# 开始训练
losses =[]
acces = []
eval_losses = []
eval_acces = []
 
for epoch in range(nums_epoch):
    train_loss = 0
    train_acc = 0
    net = net.train()
    for img , label in train_data:
        #img = img.reshape(img.size(0),-1) 
        img = Variable(img)
        label = Variable(label)
        
        # 前向传播
        out = net(img)
        loss = criterion(out,label)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 记录误差
        train_loss += loss.item()
        # 计算分类的准确率
        _,pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / img.shape[0]
       
        train_acc += acc
        
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    
    eval_loss = 0
    eval_acc = 0
    # 测试集不训练
    for img , label in test_data:
        #img = img.reshape(img.size(0),-1)
        img = Variable(img)
        label = Variable(label)
        
        out = net(img)
        
        loss = criterion(out,label)
        
        # 记录误差
        eval_loss += loss.item()
        
        _ , pred = out.max(1)
        num_correct = (pred==label).sum().item()
        acc = num_correct / img.shape[0]
        
        eval_acc += acc
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    
    print('Epoch {} Train Loss {} Train  Accuracy {} Teat Loss {} Test Accuracy {}'.format(
        epoch+1, train_loss / len(train_data),train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!


D:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Epoch 1 Train Loss 0.12345499700200416 Train  Accuracy 0.960920842217484 Teat Loss 0.03420348084624562 Test Accuracy 0.9879351265822784
Epoch 2 Train Loss 0.03286826295936644 Train  Accuracy 0.9899386993603412 Teat Loss 0.0334932597627041 Test Accuracy 0.9888251582278481
Epoch 3 Train Loss 0.022676309614899883 Train  Accuracy 0.9931203358208955 Teat Loss 0.024670037635656582 Test Accuracy 0.9921875
Epoch 4 Train Loss 0.017494460609160673 Train  Accuracy 0.994319696162047 Teat Loss 0.02609861117327708 Test Accuracy 0.991495253164557
Epoch 5 Train Loss 0.012738488631069703 Train  Accuracy 0.996018789978678 Teat Loss 0.021987379674976205 Test Accuracy 0.9931764240506329
Epoch 6 Train Loss 0.009330235840040639 Train  Accuracy 0.9969016524520256 Teat Loss 0.0213495620211481 Test Accuracy 0.9926819620253164
Epoch 7 Train Loss 0.0075988515747341615 Train  Accuracy 0.9977845149253731 Teat Loss 0.020343484291440946 Test Accuracy 0.9942642405063291
Epoch 8 Train Loss 0.006351963666287923 Train  

我们可以看到最好的测试准确度达到了99.55%，效果非常好。